In [1]:
import os
import re
import statistics
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [5]:
df = pd.read_parquet(os.path.join("data","cnb_vyber.parquet"))

In [6]:
df['rok'] = df['008'].apply(lambda x: najdi_rok(x))

In [7]:
df = df.dropna(subset=['245_a','rok'])

In [13]:
df['posledni'] = df['245_a'].apply(lambda x: x[-2:])
bordylek = df.groupby("posledni").size().nlargest(100).index.to_list()
bordylek = [x for x in bordylek if " " in x]
bordylek.append('--')
bordylek

[' /', ' :', ' =', ' ;', ' .', '--']

In [15]:
def bez_bordelu(pole):
    try:
        if pole != None:
            if isinstance(pole, list):
                print("seznam")
                pole = pole[0]
            pole = str(pole)
            for b in bordylek:
                if pole[-2:] == b:
                    pole = pole[:-2]
            if pole[-1] == ".":
                pole = pole[:-1]
            if pole[-1] == ",":
                pole = pole[:-1]
            return str(pole).strip()
        else:
            return ""
    except:
        print(pole)
        return ""

In [17]:
df['titul'] = df['245_a'].apply(lambda x: bez_bordelu(x))
df['podtitul'] = df['245_p'].apply(lambda x: bez_bordelu(x))

['Podrobný výkaz podle ústavů a poboček.'
 'Úhrnný přehled žactva odborného školství']
['Drama na moři.' 'Bonaparte /']
['Státní národ.' 'Jeho duchovní kultura.' 'Organisace státní moci /']
['Místopis Moravy.' 'Brněnský kraj.' 'Hodonský okres /']
['Část specielní.'
 'Okopaniny, zrní a semena, odpadky při zpracování zrní, pokrutiny ... (mléko, kafilerické odpadky, rybí a masová moučka atd.)']
['Slovesné druhy a literární směry.' 'Vlast, jazyk, národ.'
 'Obchod a průmysl /']
['Část specielní.' 'Krmení domácích zvířat v praxi.' 'Krmné jednotky.'
 'Technické zařízení ke krmení.' 'Pastva.'
 'Krmení koní, ... psů, dom. ptáků a ozdob. ptactva /']
['"Nové tajnosti pařížské".' 'Klub srdcových kluků /']
['Elba.' 'Západ /']
['V "Prozatímním".' 'První období Thoméovo /']
['"Nové tajnosti pařížské".' 'Tajemné dědictví /']
['V "Prozatímním".' 'Za ředitele Liegerta 1864-1865 /']
['V "Prozatímním".'
 '(Za mezivládí, po druhé za Thoméa a za pruské okupace).' '1865-1866 /']
['O podstatě vlastnictví.' 'O

In [21]:
slovni_sloupce = ['072_x','245_p','246_a','650_a','650_x','650_z','655_a']

In [103]:
import re
import unicodedata

In [123]:
def slova(radek):
    retezec = ''
    for s in slovni_sloupce:
        retezec += f" {str(radek[s])}"
    # retezec = retezec.normalize('NFKD', text)
    seznam = retezec.lower().split()
    cisty = [re.sub(r'[^a-zA-ZÁ-ŽáéěíňóřšťůúýžÄ-Žä-ž0-9]', '', s, flags=re.UNICODE) for s in seznam]
    return set(cisty)

In [129]:
df['slova'] = df.apply(lambda row: slova(row), axis=1)

In [130]:
df['slova'].sample(20)

001
np9429818                          {učebnice, vysokých, none, škol}
ck8405923                                                    {none}
np9408085                              {povídky, none, dobrodružné}
bk197901574                                                  {none}
nkc20092026642    {lékařská, vyšetření, gynekologie, contracepti...
nkc20223421920    {kultura, česko, publikace, život, vnitropolit...
nkc20142612248    {učební, učebnice, potravinářská, potravinářsk...
bk192800268                                                  {none}
bk195202270                                                  {none}
nkc20152661437    {none, personnel, personální, řízení, monograf...
cpk19960115957    {dobrodružná, none, vědeckofantastické, milost...
cpe20162792213    {college, studie, cdrom, foreign, none, zahran...
cpk19990656983    {picture, books, literatura, children, none, p...
nkc20112188581    {americká, fiction, american, americké, none, ...
nkc20213377276    {, novels, publikace, pro,

In [131]:
jen_slova = df[['slova','rok']].explode('slova')

In [133]:
vsechna_slova = set(jen_slova['slova'].drop_duplicates().to_list())

In [139]:
len(vsechna_slova)

137432

In [132]:
jen_slova.groupby('slova').size().nlargest(20)

slova
none          849098
a             176602
próza         106729
česko         105058
publikace     104566
czechia        81902
literatura     76741
učebnice       75464
and            74652
česká          71113
příručky       70575
škol           67722
romány         64285
pro            64114
czech          62519
               60977
děti           57190
učební         56701
předměty       54377
osnovy         53862
dtype: int64

In [167]:
rok = 1949
poroce = jen_slova[jen_slova['rok'] == rok]['slova'].drop_duplicates().to_list()
predrokem = set(jen_slova[jen_slova['rok'] < rok]['slova'].drop_duplicates().to_list())
nejsou = []
for p in poroce:
    if p not in predrokem:
        nejsou.append(p)

In [169]:
jen_slova[jen_slova['slova'].isin(nejsou)].groupby('slova').size().nlargest(10)

slova
řídicí                512
automatizační         469
dokumentace           353
plastů                287
gumárenský            272
makromolekulárních    263
audit                 226
skládanky             218
auditing              193
welding               157
dtype: int64